# 1. Introdução
O notebook abaixo apresenta e implementa o spot-checking de modelos preditivos supervisionados, desenvolvido para primeiro trabalho da disciplina Aprendizado de Máquina da Universidade Federal do Rio Grande do Sul (2024/2).

Neste trabalho, buscamos analisar a relação de diversos fatores, como gênero e notas do primeiro semestre, com a taxa de desistência de alunos. No modelo abaixo usamos o dataset carregado nesse notebook, analisamos quais os fatores que de fato influenciam na desistência dos alunos e possibilitamos que inputs personalizados sejam adicionados ao modelo para que seja calculado a probabilidade de um aluno desistir do curso.


## Setup

In [3]:
!pip install pandas plotly matplotlib seaborn scikit-learn xgboost

In [4]:
# Módulo para leitura e manipulação dos dados
import pandas as pd

# Módulo para manipulação de arrays e matrizes
import numpy as np

# Módulos para visualização de dados e plotagem de gráficos
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Módulos específicos da sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Biblioteca com algoritmos específicos de machine learning
from xgboost import XGBClassifier

# Módulo para balanceamento de classes
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import optuna
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

ModuleNotFoundError: No module named 'pkg_resources'

# 1. Carregamento dos dados

Dataset pré-processado no notebook [t1-spot-checking.ipynb](./t1-spot-checking.ipynb)

---
Dataset obtido em https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention/data

Original: https://zenodo.org/records/5777340#.Y7FJotJBwUE

In [ ]:
data = pd.read_csv("../data/clean-dataset.csv")

In [ ]:
# Separa atributos preditivos e atributo alvo
X = data.drop('Target', axis=1)
y = data['Target']

In [ ]:
# Algoritmos selecionados para treinamento
dtree = DecisionTreeClassifier(random_state=0)
dtree2 = DecisionTreeClassifier(random_state=0, max_depth=10)
rfc_gini = RandomForestClassifier(random_state=2)
rfc_entropy = RandomForestClassifier(random_state=2, criterion='entropy')
lr = LogisticRegression(random_state=42)
knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_5 = KNeighborsClassifier(n_neighbors=5)
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1, random_state=0, algorithm='SAMME')
svmachine = svm.SVC(kernel='linear',probability=True)

algo_dict = {'Decision Tree': dtree, 'Decision Tree Max depth 5': dtree2, 'Random Forest gini': rfc_gini, 'Random Forest entropy': rfc_entropy, 'Logistic Regression': lr, '3-Nearest Neighbors': knn_3, '5-Nearest Neighbors': knn_5, 'AdaBoost': abc, 'SVM': svmachine}

In [ ]:
# Referências
# https://machinelearningmastery.com/spot-check-machine-learning-algorithms-in-python/
# https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
def make_pipeline(model):
    steps = list()

    steps.append(('Normalização', StandardScaler()))
    steps.append(('Balanceamento da classe minoritária', SMOTE(sampling_strategy='minority')))
    steps.append(('Modelo', model))

    # Cria a pipeline
    pipe = Pipeline(steps=steps)

    return pipe

In [ ]:
# Define hyperparameter grids for each model
def define_grid_search_params(model_name):
    if model_name == 'Random Forest':
        return {
            'Modelo__n_estimators': [50, 100, 150],
            'Modelo__max_depth': [10, 20, 30]
        }
    elif model_name == 'Decision Tree':
        return {
            'Modelo__max_depth': [5, 10, 20, 30]
        }
    elif model_name == 'Logistic Regression':
        return {
            'Modelo__C': [0.01, 0.1, 1, 10, 100]
        }
    elif model_name == 'KNN':
        return {
            'Modelo__n_neighbors': [3, 5, 7, 10]
        }
    elif model_name == 'AdaBoost':
        return {
            'Modelo__n_estimators': [50, 100, 150],
            'Modelo__learning_rate': [0.01, 0.1, 1]
        }
    elif model_name == 'SVM':
        return {
            'Modelo__C': [0.01, 0.1, 1, 10],
            'Modelo__kernel': ['linear', 'rbf']
        }

In [ ]:
# Nested cross-validation with GridSearchCV
def nested_cv_with_gridsearch(X, y, models, outer_folds=5, inner_folds=3, metric='f1'):
    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for model_name, model in models.items():
        print(f"Optimizing and evaluating model: {model_name}")

        # Define parameter grid
        param_grid = define_grid_search_params(model_name)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # Create pipeline
            pipeline = make_pipeline(model)

            # Set up GridSearchCV
            grid_search = GridSearchCV(
                estimator=pipeline,
                param_grid=param_grid,
                cv=StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42),
                scoring=metric,
                n_jobs=-1
            )

            # Fit GridSearchCV
            grid_search.fit(X_train, y_train)

            # Use the best model from the grid search
            best_pipeline = grid_search.best_estimator_

            # Evaluate on the test set
            y_pred = best_pipeline.predict(X_test)
            metrics = {
                'Model': model_name,
                'F1 Score': f1_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'ROC AUC': roc_auc_score(y_test, best_pipeline.predict_proba(X_test)[:, 1])
            }
            results.append(metrics)

    return pd.DataFrame(results)

In [ ]:
# Execute nested CV with GridSearch optimization
results_df = nested_cv_with_gridsearch(X, y, algo_dict, outer_folds=5, inner_folds=3, metric='f1')

# Display results as a DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Model Performance Results with GridSearch", dataframe=results_df)

In [ ]:
# Define the space for hyperparameter optimization
def define_search_space(model_name):
    if model_name == 'Random Forest':
        return {
            'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
            'max_depth': hp.quniform('max_depth', 5, 50, 1),
        }
    elif model_name == 'Decision Tree':
        return {
            'max_depth': hp.quniform('max_depth', 5, 50, 1),
        }
    elif model_name == 'Logistic Regression':
        return {
            'C': hp.loguniform('C', np.log(1e-4), np.log(1e2)),
        }
    elif model_name == 'KNN':
        return {
            'n_neighbors': hp.quniform('n_neighbors', 3, 20, 1),
        }
    elif model_name == 'AdaBoost':
        return {
            'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
            'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(2)),
        }
    elif model_name == 'SVM':
        return {
            'C': hp.loguniform('C', np.log(1e-4), np.log(1e2)),
            'kernel': hp.choice('kernel', ['linear', 'rbf']),
        }

In [ ]:
# Nested cross-validation with Hyperopt
def nested_cv_with_hyperopt(X, y, models, outer_folds=5, inner_folds=3, metric='f1'):
    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for model_name, model in models.items():
        print(f"Optimizing and evaluating model: {model_name}")

        def objective(params):
            # Create the pipeline with the model
            pipeline = make_pipeline(model)
            # Update model-specific hyperparameters
            pipeline.set_params(**{f'Modelo__{k}': int(v) if isinstance(v, float) and v.is_integer() else v for k, v in params.items()})

            # Perform cross-validation
            inner_cv = StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42)
            scores = cross_val_score(pipeline, X_train, y_train, cv=inner_cv, scoring=metric, n_jobs=-1)
            return {'loss': -np.mean(scores), 'status': STATUS_OK}

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # Define the search space for the current model
            search_space = define_search_space(model_name)

            # Run the optimization
            trials = Trials()
            best_params = fmin(
                fn=objective,
                space=search_space,
                algo=tpe.suggest,
                max_evals=20,
                trials=trials,
            )

            # Convert the best_params to the correct format for the pipeline
            best_params = {f'Modelo__{k}': int(v) if isinstance(v, float) and v.is_integer() else v for k, v in best_params.items()}

            # Train the pipeline with the best parameters
            pipeline = make_pipeline(model)
            pipeline.set_params(**best_params)
            pipeline.fit(X_train, y_train)

            # Evaluate on the test set
            y_pred = pipeline.predict(X_test)
            metrics = {
                'Model': model_name,
                'F1 Score': f1_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'ROC AUC': roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1])
            }
            results.append(metrics)

    return pd.DataFrame(results)

In [ ]:
# Execute nested CV with Hyperopt optimization
results_df = nested_cv_with_hyperopt(X, y, algo_dict, outer_folds=5, inner_folds=3, metric='f1')

# Display results as a DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Model Performance Results with Hyperopt", dataframe=results_df)

In [ ]:
def nested_cv_with_optuna(X, y, models, outer_folds=5, inner_folds=3, metric='f1'):
    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for model_name, model in models.items():
        print(f"Optimizing and evaluating model: {model_name}")

        def objective(trial):
            params = {}
            if model_name == 'Random Forest':
                params = {
                    'Modelo__n_estimators': trial.suggest_int('n_estimators', 50, 300),
                    'Modelo__max_depth': trial.suggest_int('max_depth', 5, 50)
                }
            elif model_name == 'Decision Tree':
                params = {
                    'Modelo__max_depth': trial.suggest_int('max_depth', 5, 50)
                }
            elif model_name == 'Logistic Regression':
                params = {
                    'Modelo__C': trial.suggest_loguniform('C', 1e-4, 1e2)
                }
            elif model_name == 'KNN':
                params = {
                    'Modelo__n_neighbors': trial.suggest_int('n_neighbors', 3, 20)
                }
            elif model_name == 'AdaBoost':
                params = {
                    'Modelo__n_estimators': trial.suggest_int('n_estimators', 50, 300),
                    'Modelo__learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 2)
                }
            elif model_name == 'SVM':
                params = {
                    'Modelo__C': trial.suggest_loguniform('C', 1e-4, 1e2),
                    'Modelo__kernel': trial.suggest_categorical('kernel', ['linear', 'rbf'])
                }

            pipeline = make_pipeline(model)
            pipeline.set_params(**params)
            inner_cv = StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42)

            return cross_val_score(pipeline, X_train, y_train, cv=inner_cv, scoring=metric, n_jobs=-1).mean()

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=20)

            # Add the 'Modelo__' prefix to all best_params keys
            best_params = {f'Modelo__{key}': value for key, value in study.best_params.items()}

            pipeline = make_pipeline(model)
            pipeline.set_params(**best_params)
            pipeline.fit(X_train, y_train)

            # Evaluate on test set
            y_pred = pipeline.predict(X_test)
            metrics = {
                'Model': model_name,
                'F1 Score': f1_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'ROC AUC': roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1])
            }
            results.append(metrics)

    return pd.DataFrame(results)

In [ ]:
# Execute nested CV with Optuna optimization
results_df = nested_cv_with_optuna(X, y, algo_dict, outer_folds=5, inner_folds=3, metric='f1')

# Display results as a DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Model Performance Results", dataframe=results_df)

[I 2024-12-07 00:02:12,642] A new study created in memory with name: no-name-50eea8a6-400d-4428-8ef8-9c9f36387c60


Optimizing and evaluating model: Decision Tree


[I 2024-12-07 00:02:13,754] Trial 0 finished with value: 0.7042123926476935 and parameters: {'max_depth': 19}. Best is trial 0 with value: 0.7042123926476935.
[I 2024-12-07 00:02:14,373] Trial 1 finished with value: 0.6955922377121695 and parameters: {'max_depth': 39}. Best is trial 0 with value: 0.7042123926476935.
[I 2024-12-07 00:02:15,115] Trial 2 finished with value: 0.7265002052568476 and parameters: {'max_depth': 10}. Best is trial 2 with value: 0.7265002052568476.
[I 2024-12-07 00:02:15,744] Trial 3 finished with value: 0.6923257399652716 and parameters: {'max_depth': 23}. Best is trial 2 with value: 0.7265002052568476.
[I 2024-12-07 00:02:16,383] Trial 4 finished with value: 0.7151666595531897 and parameters: {'max_depth': 12}. Best is trial 2 with value: 0.7265002052568476.
[I 2024-12-07 00:02:17,031] Trial 5 finished with value: 0.7049937980058204 and parameters: {'max_depth': 32}. Best is trial 2 with value: 0.7265002052568476.
[I 2024-12-07 00:02:17,652] Trial 6 finished w

Optimizing and evaluating model: Decision Tree Max depth 5


[I 2024-12-07 00:02:21,803] Trial 8 finished with value: 0.7351932811810591 and parameters: {}. Best is trial 7 with value: 0.7364528219901073.
[I 2024-12-07 00:02:21,826] Trial 9 finished with value: 0.7329549523044544 and parameters: {}. Best is trial 7 with value: 0.7364528219901073.
[I 2024-12-07 00:02:21,849] Trial 10 finished with value: 0.730397651115831 and parameters: {}. Best is trial 7 with value: 0.7364528219901073.
[I 2024-12-07 00:02:21,872] Trial 11 finished with value: 0.7296079107361136 and parameters: {}. Best is trial 7 with value: 0.7364528219901073.
[I 2024-12-07 00:02:21,895] Trial 12 finished with value: 0.7297141099171557 and parameters: {}. Best is trial 7 with value: 0.7364528219901073.
[I 2024-12-07 00:02:21,918] Trial 13 finished with value: 0.7307523972606562 and parameters: {}. Best is trial 7 with value: 0.7364528219901073.
[I 2024-12-07 00:02:21,941] Trial 14 finished with value: 0.7310830924891452 and parameters: {}. Best is trial 7 with value: 0.736452

...

Optimizing and evaluating model: AdaBoost


[I 2024-12-07 00:03:15,357] Trial 0 finished with value: 0.7682161154280114 and parameters: {'n_estimators': 250, 'learning_rate': 0.0728867765360118}. Best is trial 0 with value: 0.7682161154280114.
C:\Users\Matheus\AppData\Local\Temp\ipykernel_28488\2421095699.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'Modelo__learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 2)
[I 2024-12-07 00:03:15,516] Trial 1 finished with value: 0.7602769624091407 and parameters: {'n_estimators': 126, 'learning_rate': 1.0292128000483027}. Best is trial 0 with value: 0.7682161154280114.
C:\Users\Matheus\AppData\Local\Temp\ipykernel_28488\2421095699.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_fl

Optimizing and evaluating model: SVM


C:\Users\Matheus\AppData\Local\Temp\ipykernel_28488\2421095699.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'Modelo__C': trial.suggest_loguniform('C', 1e-4, 1e2),
[I 2024-12-07 00:03:39,752] Trial 0 finished with value: 0.7437747766880353 and parameters: {'C': 0.03772548792712767, 'kernel': 'linear'}. Best is trial 0 with value: 0.7437747766880353.
C:\Users\Matheus\AppData\Local\Temp\ipykernel_28488\2421095699.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'Modelo__C': trial.suggest_loguniform('C', 1e-4, 1e2),
[I 2024-12-07 00:03:40,330] Trial 1 finished with value: 0.7491489525638045 and parameters: {'C': 4.553194564617522, 'kernel': 'linear'}. Best is tr

ModuleNotFoundError: No module named 'ace_tools'